In [1]:
import pandas as pd
import numpy as np
import requests

# Getting the raw GDP data

### very helpful thread:

        https://www.reddit.com/r/learnpython/comments/h8wg56/web_scraping_an_interactive_table/

In [2]:
# read the pre-downloaded table
gdp_df = pd.read_csv('COUNTY_GDP9_2021table.csv', encoding='ISO-8859-1', header=None)

# set the header
headers = ['AreaName', 'St', 'cfips', 'Ind1', 'Ind2', 'IndName', 
            'RGDP_2019', 'RGDP_2019_pct', 
            'RGDP_2020', 'RGDP_2020_pct', 
            'RGDP_2021', 'RGDP_2021_pct',
            'RGDP_19_to_21', 'RGDP_19_to_21_pct',]
gdp_df.columns = headers

/var/folders/8v/b1l_n51d2_b4pkp_5kxnxwhh0000gn/T/ipykernel_73844/3815737122.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  gdp_df = pd.read_csv('COUNTY_GDP9_2021table.csv', encoding='ISO-8859-1', header=None)


In [3]:
# note here we have the same cfips problem as in the previous notebook
gdp_df["cfips_len"] = gdp_df["cfips"].apply(lambda x: len(str(x)))
gdp_df["cfips"] = gdp_df["cfips"].apply(lambda x: str(x))
gdp_df.loc[gdp_df.cfips_len==4, "cfips"] = "0" + gdp_df.loc[gdp_df.cfips_len==4, "cfips"]
gdp_df.drop(columns=["cfips_len"], inplace=True)

In [4]:
# show the first 5 rows
gdp_df.head()

,AreaName,St,cfips,Ind1,Ind2,IndName,RGDP_2019,RGDP_2019_pct,RGDP_2020,RGDP_2020_pct,RGDP_2021,RGDP_2021_pct,RGDP_19_to_21,RGDP_19_to_21_pct
0,Alabama,AL,01000,1,...,All industry total,203432712.0,100.00,199880770.0,100.00,209979322.0,100.00,6546610.0,3.22
1,Alabama,AL,01000,2,...,. Private industries,171431441.0,84.27,167545502.0,83.82,177627942.0,84.59,6196501.0,3.61
2,Alabama,AL,01000,3,11,".. Agriculture, forestry, fishing and hunting",2666703.0,1.31,2190311.0,1.10,2870961.0,1.37,204258.0,7.66
3,Alabama,AL,01000,6,21,".. Mining, quarrying, and oil and gas extrac...",2500591.0,1.23,2373533.0,1.19,2196814.0,1.05,-303777.0,-12.15
4,Alabama,AL,01000,10,22,.. Utilities,5581552.0,2.74,5668429.0,2.84,5320995.0,2.53,-260557.0,-4.67


In [5]:
# check unique values for the IndName column
gdp_df.IndName.unique()

array(['All industry total', '.  Private industries',
       '..   Agriculture, forestry, fishing and hunting',
       '..   Mining, quarrying, and oil and gas extraction',
       '..   Utilities', '..   Construction', '..   Manufacturing',
       '...    Durable goods manufacturing',
       '...    Nondurable goods manufacturing', '..   Wholesale trade',
       '..   Retail trade', '..   Transportation and warehousing',
       '..   Information',
       '..   Finance, insurance, real estate, rental, and leasing',
       '...    Finance and insurance',
       '...    Real estate and rental and leasing',
       '..   Professional and business services',
       '...    Professional, scientific, and technical services',
       '...    Management of companies and enterprises',
       '...    Administrative and support and waste management and remediation services',
       '..   Educational services, health care, and social assistance',
       '...    Educational services',
       '...    H

# Load Previous Given Data

### Given Census Data

In [6]:
# load the census data, first column of string type, other of float type
dtypes = {'cfips': str}
census_df = pd.read_csv('../DATA/census_given.csv', header=0, dtype=dtypes)
census_df.head()

,cfips,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
0,01001,76.6,78.9,80.6,82.7,85.5,14.5,15.9,16.1,16.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,01003,74.5,78.1,81.8,85.1,87.9,20.4,20.7,21.0,20.2,...,1.4,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0
2,01005,57.2,60.4,60.5,64.6,64.6,7.6,7.8,7.6,7.3,...,0.5,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0
3,01007,62.0,66.1,69.2,76.1,74.6,8.1,7.6,6.5,7.4,...,1.2,1.4,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0
4,01009,65.8,68.5,73.0,79.6,81.0,8.7,8.1,8.6,8.9,...,1.3,1.4,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0


In [7]:
census_df['cfips'].nunique()

3142

### cfips to county name data

In [8]:
# load the cfips to county name mapping, first column of string type
dtypes = {'cfips': str, 'county_name': str, 'state': str}
cfips_to_county = pd.read_csv('../DATA/cfips_to_county_name.csv', header=0, dtype=dtypes)
cfips_to_county.head()

,cfips,county_name,state
0,01001,Autauga County,AL
1,01003,Baldwin County,AL
2,01005,Barbour County,AL
3,01007,Bibb County,AL
4,01009,Blount County,AL


In [9]:
cfips_to_county['cfips'].nunique()

3143

### data to be filled in

In [10]:
# load the empty cfips dataframe
dtypes = {'cfips': str}
cfips_df = pd.read_csv('../DATA/cfips.csv', header=0, dtype=dtypes)
cfips_df.head()

,cfips
0,01001
1,01003
2,01005
3,01007
4,01009


In [11]:
cfips_df['cfips'].nunique()

3142

# adding real GDP into county level data

In [12]:
# select total GDP for all counties
gdp_df_proc = gdp_df.copy()
gdp_df_proc = gdp_df_proc.loc[gdp_df_proc.IndName == 'All industry total', :]

# keeping only 4 columns: cfips, RGDP_2019, RGDP_2020, RGDP_2021
gdp_df_proc = gdp_df_proc[['cfips', 'RGDP_2019', 'RGDP_2020', 'RGDP_2021']]
gdp_df_proc.head()

# join the cfips_df with gdp_df_procdata frame on the cfips column of the cfips_df
new_cfips_df = pd.merge(cfips_df, gdp_df_proc, on='cfips', how='left')
new_cfips_df.head()

,cfips,RGDP_2019,RGDP_2020,RGDP_2021
0,01001,1543336.0,1525591.0,1502153.0
1,01003,7132205.0,7142139.0,7830237.0
2,01005,728447.0,699829.0,709459.0
3,01007,380403.0,398101.0,392249.0
4,01009,937239.0,883216.0,997835.0


In [13]:
# save the new cfips_df to a csv file
new_cfips_df.to_csv('Scrapped_GDP_DATA/rGDP_19_20_21.csv', index=False)

# County's Sector Percent GDP 

### Sectors of interest (ZZP handpicked): 

        Agri: 
                ..   Agriculture, forestry, fishing and hunting
        Mine:
                ..   Mining, quarrying, and oil and gas extraction
                (Private Sector Mining Business)
        Retl:
                ..   Retail trade
        Manu:
                ..   Manufacturing
        Info:
                ..   Information
        FinS:
                ..   Finance, insurance, real estate, rental, and leasing
        EduS:
                ..   Educational services, health care, and social assistance
        EntS:
                ..   Arts, entertainment, recreation, accommodation, and food services
        Gvmt:
                . Government and government enterprises
        GoMn:
                . Natural resources and mining
                (Public Sector Mining Business)
        GoTr:
                . Transportation and utilities
        GoMf:
                . Manufacturing and information
                

In [14]:
feature_dict = {
    'Agri': '..   Agriculture, forestry, fishing and hunting',
    'Mine': '..   Mining, quarrying, and oil and gas extraction',
    'Retl': '..   Retail trade',
    'Manu': '..   Manufacturing',
    'Info': '..   Information',
    'FinS': '..   Finance, insurance, real estate, rental, and leasing',
    'EduS': '..   Educational services, health care, and social assistance',
    'EntS': '..   Arts, entertainment, recreation, accommodation, and food services',
    'Gvmt': '. Government and government enterprises',
    'GoMn': '. Natural resources and mining',
    'GoTr': '. Transportation and utilities',
    'GoMf': '. Manufacturing and information'
}
input_list = []
for key, value in feature_dict.items():
    input_list.append([key, value])

In [15]:
def add_sector_pct_data(new_cfips_df, abriviation, ind_string_name):
    gdp_df_proc = gdp_df.copy()
    gdp_df_proc = gdp_df_proc.loc[gdp_df_proc.IndName == ind_string_name, :]
    gdp_df_proc = gdp_df_proc[['cfips', 'RGDP_2019_pct', 'RGDP_2020_pct', 'RGDP_2021_pct']]
    gdp_df_proc.columns = ['cfips', f'GDP_{abriviation}_pct_2019', f'GDP_{abriviation}_pct_2020', f'GDP_{abriviation}_pct_2021']
    new_new_cfips_df = pd.merge(new_cfips_df, gdp_df_proc, on='cfips', how='left')
    return new_new_cfips_df

In [16]:
new_cfips_df = add_sector_pct_data(cfips_df, input_list[0][0], input_list[0][1])
new_cfips_df.head()

,cfips,GDP_Agri_pct_2019,GDP_Agri_pct_2020,GDP_Agri_pct_2021
0,01001,2.38,2.64,1.59
1,01003,0.33,0.55,0.43
2,01005,9.40,5.69,9.57
3,01007,1.24,1.20,0.97
4,01009,8.56,1.58,10.67


In [17]:
for i in range(1, len(input_list)):
    new_cfips_df = add_sector_pct_data(new_cfips_df, input_list[i][0], input_list[i][1])
    
new_cfips_df.head()

,cfips,GDP_Agri_pct_2019,GDP_Agri_pct_2020,GDP_Agri_pct_2021,GDP_Mine_pct_2019,GDP_Mine_pct_2020,GDP_Mine_pct_2021,GDP_Retl_pct_2019,GDP_Retl_pct_2020,GDP_Retl_pct_2021,...,GDP_Gvmt_pct_2021,GDP_GoMn_pct_2019,GDP_GoMn_pct_2020,GDP_GoMn_pct_2021,GDP_GoTr_pct_2019,GDP_GoTr_pct_2020,GDP_GoTr_pct_2021,GDP_GoMf_pct_2019,GDP_GoMf_pct_2020,GDP_GoMf_pct_2021
0,01001,2.38,2.64,1.59,1.04,1.16,1.20,6.44,6.59,6.77,...,10.92,3.43,3.82,2.81,21.88,23.69,21.98,17.74,17.13,17.92
1,01003,0.33,0.55,0.43,0.19,0.18,0.18,12.42,12.17,11.39,...,9.76,0.52,0.78,0.63,2.15,2.29,2.10,8.86,7.78,7.93
2,01005,9.40,5.69,9.57,4.78,3.58,3.22,7.85,7.70,7.44,...,14.65,14.28,9.47,12.77,5.56,5.66,-0.00,26.88,28.97,26.63
3,01007,1.24,1.20,0.97,0.31,0.34,0.32,7.35,7.21,7.37,...,20.84,1.57,1.55,1.28,3.01,5.17,4.96,-0.00,-0.00,-0.00
4,01009,8.56,1.58,10.67,0.19,0.24,0.12,7.41,7.84,6.96,...,13.89,9.17,1.99,11.43,-0.00,-0.00,-0.00,13.94,13.93,13.36


In [18]:
new_cfips_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 0 to 3141
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cfips              3142 non-null   object 
 1   GDP_Agri_pct_2019  3089 non-null   float64
 2   GDP_Agri_pct_2020  3089 non-null   float64
 3   GDP_Agri_pct_2021  3089 non-null   float64
 4   GDP_Mine_pct_2019  3089 non-null   float64
 5   GDP_Mine_pct_2020  3089 non-null   float64
 6   GDP_Mine_pct_2021  3089 non-null   float64
 7   GDP_Retl_pct_2019  3089 non-null   float64
 8   GDP_Retl_pct_2020  3089 non-null   float64
 9   GDP_Retl_pct_2021  3089 non-null   float64
 10  GDP_Manu_pct_2019  3089 non-null   float64
 11  GDP_Manu_pct_2020  3089 non-null   float64
 12  GDP_Manu_pct_2021  3089 non-null   float64
 13  GDP_Info_pct_2019  3089 non-null   float64
 14  GDP_Info_pct_2020  3089 non-null   float64
 15  GDP_Info_pct_2021  3089 non-null   float64
 16  GDP_FinS_pct_2019  3089 

In [19]:
# save the new cfips_df to a csv file
new_cfips_df.to_csv('Scrapped_GDP_DATA/sectorGDPpct_19_20_21.csv', index=False)